In [1]:
!pip install markdown langchain pdfminer.six


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00


In [3]:
!pip install -U sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=adcbe41a294b6cf25c04fe0d8f31d67b77c410241d452e53a6adfbbab727db4a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
!pip install InstructorEmbedding

In [16]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00


In [25]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3510601 sha256=be0ff1c7b6d6f4aa78668f01db53fe0578d22390ed5066237d9618af57f5f138
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python


In [5]:
import os
import re
import markdown
from pdfminer.high_level import extract_text as extract_text_from_pdf
from io import StringIO
from html.parser import HTMLParser
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
# HTML Stripper class to clean HTML tags from the text
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = StringIO()

    def handle_data(self, d):
        self.text.write(d)

    def get_data(self):
        return self.text.getvalue()


In [7]:
def strip_tags(html):
    """Remove HTML tags from a string."""
    s = MLStripper()
    s.feed(html)
    return s.get_data()


In [8]:
def clean_markdown(text):
    """Clean Markdown syntax from text."""
    # Remove Markdown URL links
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)
    # Remove bold and italic text markers
    text = re.sub(r'\*\*([^*]+)\*\*', r'\1', text)
    text = re.sub(r'\*([^*]+)\*', r'\1', text)
    text = re.sub(r'__([^_]+)__', r'\1', text)
    text = re.sub(r'_([^_]+)_', r'\1', text)
    # Remove images and their references
    text = re.sub(r'!\[[^\]]]*]\([^)]*\)', '', text)
    # Remove headers markers
    text = re.sub(r'#+\s?', '', text)
    # Remove other Markdown syntax as needed (e.g., tables, bullet points)
    text = re.sub(r'\|', ' ', text)
    text = re.sub(r'-{2,}', '', text)
    text = re.sub(r'\n{2,}', '\n', text)  # Remove extra newlines
    return text


In [9]:
def extract_text_from_md(md_path):
    """Extract and clean text from a Markdown file."""
    with open(md_path, "r", encoding="utf-8") as file:
        md_content = file.read()
        html = markdown.markdown(md_content)
        text = strip_tags(html)
        return clean_markdown(text)


In [10]:
def extract_text_from_file(file_path):
    """Extract text from a file based on its extension."""
    if file_path.endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith('.md'):
        return extract_text_from_md(file_path)
    elif file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    else:
        return "Unsupported file format."


In [11]:
# Directory containing documents to process
directory = r'/content/drive/MyDrive/Papers/'


In [12]:
# Parameters for text splitting
chunk_size = 1200
chunk_overlap = 300

# List to store all document chunks
all_docs = []
allowed_extensions = ['.md', '.pdf', '.txt']

# Process each file in the directory
for root, dirs, files in os.walk(directory):
    for filename in files:
        # Get the file extension
        _, file_extension = os.path.splitext(filename)
        if file_extension in allowed_extensions:
            file_path = os.path.join(root, filename)  # Full path of the file

            # Remove the ".md", ".pdf" or ".txt" extension from the file name
            file_name_without_extension = os.path.splitext(filename)[0]

            # Open and read the file
            file_content = extract_text_from_file(file_path)

            # Split the text into chunks
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
            docs = text_splitter.split_text(file_content)

            for i, chunk in enumerate(docs):
                # Define metadata for each chunk (you can customize this)
                metadata = {
                    "File Name": file_name_without_extension,
                    "Chunk Number": i + 1,
                }

                # Create a header with metadata and file name
                header = f"File Name: {file_name_without_extension}\n"
                for key, value in metadata.items():
                    header += f"{key}: {value}\n"

                # Combine header, file name, and chunk content
                chunk_with_header = header + file_name_without_extension + "\n" + chunk
                all_docs.append(chunk_with_header)

            print(f"Processed: {filename}")

Processed: 18.Regularized Latent Semantic Indexing.pdf
Processed: 17.Deep Belief Nets for Topic Modeling.pdf
Processed: 16.Modeling Documents with a Deep Boltzmann Machine.pdf
Processed: External Evaluation of Topic Models.pdf
Processed: Distributed Algorithms for Topic Models.pdf
Processed: Online Learning for Matrix Factorization and Sparse Coding.pdf
Processed: A Rotation Test to Verify Latent Structure.pdf
Processed: Why Does Unsupervised Pre-training Help Deep Learning.pdf
Processed: Paper_21-A_Survey_of_Topic_Modeling_in_Text_Mining.pdf
Processed: SURVEY-1.pdf
Processed: 13A SURVEY ON TOPIC MODELLING.pdf
Processed: TopicModeling-ASM.pdf
Processed: Dynamic Topic Models.pdf
Processed: A Topic Modeling Toolbox Using Belief Propagation.pdf
Processed: 01.Indexing by latent semantic analysis.pdf
Processed: 02.Probabilistic latent semantic indexing.pdf
Processed: 04.Hierarchical topic models and the nested Chinese restaurant process.pdf
Processed: 05.Dynamic Topic Models.pdf
Processed: 

In [27]:
len(all_docs)

4521

In [17]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings


In [18]:
# Initialize HuggingFaceInstructEmbeddings
model_name = "hkunlp/instructor-large"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
hf_embedding = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Embed and index all the documents using FAISS
db = FAISS.from_texts(all_docs, hf_embedding)

# Save the indexed data locally
db.save_local("faiss_AiDoc")


load INSTRUCTOR_Transformer
max_seq_length  512


In [20]:
# Load the FAISS index from local storage
db = FAISS.load_local("faiss_AiDoc", embeddings=hf_embedding,allow_dangerous_deserialization=True)


In [21]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (
    StreamingStdOutCallbackHandler
)

In [22]:
# Template for question-answer prompt
template = """Question: {question} \n\nAnswer:"""
# Initialize prompt template and callback manager
prompt = PromptTemplate(template=template, input_variables=["question"])
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


In [23]:
# Define the local path to the Llama2 model download
model_path = "/content/drive/MyDrive/Llama/llama-2-7b-chat.Q6_K.gguf"


In [32]:
# Initialize LlamaCpp model
llm = LlamaCpp(model_path=model_path, temperature=0.2, max_tokens=4095, top_p=1, callback_manager=callback_manager, n_ctx=6000)
# Create LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /content/drive/MyDrive/Llama/llama-2-7b-chat.Q6_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.hea

In [36]:
# Define a query to search the indexed documents
query = "What is topic modeling?"
# Search for semantically similar chunks and return the top 5 chunks
search = db.similarity_search(query, k=5)


In [37]:
# Define a template for generating a final prompt
template = '''Context: {context}
Based on the Context, please answer the following question:
Question: {question}
Provide an answer based on the context only, without using general knowledge. The answer
should be derived directly from the context provided.
Please correct any grammatical errors for improved readability.
If the context does not contain relevant information to answer the question, state that the
answer is not available in the given context.
Please include the source title of the information as a reference of how you arrive at your
answer. '''


In [38]:
# Create a prompt template
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

# Format the final prompt with the query and search results
final_prompt = prompt.format(question=query, context=search)

# Run LLMChain to generate an answer based on the context
llm_chain.run(final_prompt)

Llama.generate: 18 prefix-match hit, remaining 1729 prompt tokens to eval


 Based on the context provided, topic modeling refers to a group of machine learning algorithms that infer the latent structure behind a collection of documents. The intuition behind topic models is that each document is comprised of some ‘topics’ or themes, and these topics are understood as collections of words that represent the topic as a whole. Topic modeling techniques are statistical methods that analyze the words of documents and discover the topics that run through them. Knowing these topics would help us in many ways, such as understanding the document's utility based on its content, clustering documents based on similar topics, and identifying the depth of a particular topic or multiple topics with different using mathematical frameworks.

llama_perf_context_print:        load time =  840426.34 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  2529 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   142 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  975558.50 ms /  2671 tokens


" Based on the context provided, topic modeling refers to a group of machine learning algorithms that infer the latent structure behind a collection of documents. The intuition behind topic models is that each document is comprised of some ‘topics’ or themes, and these topics are understood as collections of words that represent the topic as a whole. Topic modeling techniques are statistical methods that analyze the words of documents and discover the topics that run through them. Knowing these topics would help us in many ways, such as understanding the document's utility based on its content, clustering documents based on similar topics, and identifying the depth of a particular topic or multiple topics with different using mathematical frameworks."